# CONVERTING KERAS/TF MODEL TO ONNX FORMAT

In [1]:
import os
import cv2
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sklearn
from transformers import ViTFeatureExtractor, TFViTModel

/opt/homebrew/Caskroom/miniforge/base/envs/HuggingFaceTransformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONFIGURATION = {
    "CLASS_NAMES" : ['angry', 'happy', 'sad'],
    "BATCH_SIZE" : 32,
    "IMAGE_SIZE" : 224,
    "LEARNING_RATE" : 0.01,
    "N_EPOCHS" : 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 128,
    "N_DENSE_2" : 128,
    "NUM_CLASSES" : 3,
    "PATCH_SIZE" : 16,
}

trainDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/train"
testDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test"

trainDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='training',
)

valDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='validation',
)

testDataset = tf.keras.utils.image_dataset_from_directory(
    testDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=None,
    subset=None,
)

trainDataset = trainDataset.prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.prefetch(tf.data.AUTOTUNE)

Found 6799 files belonging to 3 classes.
Using 5440 files for training.
Found 6799 files belonging to 3 classes.
Using 1359 files for validation.
Found 2278 files belonging to 3 classes.


2025-02-02 18:13:24.646975: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-02 18:13:24.647002: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-02 18:13:24.647009: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1738548804.647048 18106103 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1738548804.647085 18106103 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Loading Keras Model

In [3]:
ViTModel = tf.keras.models.load_model("Models/EmotionDetectionViT.keras")
ViTModel.evaluate(testDataset)

2025-02-02 18:13:40.598135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


72/72 [==============================] - 39s 516ms/step - loss: 0.5523 - accuracy: 0.8749 - top_k_accuracy: 0.9680


[0.5522751212120056, 0.8748902678489685, 0.9679543375968933]

## Installing ONNX Tools

In [26]:
# ! pip install -U tf2onnx
# ! pip install -U keras2onnx
# ! pip install -U onnxruntime

## Conversion

In [5]:
import tf2onnx
import onnxruntime as rt

In [6]:
onnxModel, _ = tf2onnx.convert.from_keras(ViTModel, output_path="Models/EmotionDetectionViT.onnx")

I0000 00:00:1738548908.728408 18106103 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1738548908.728493 18106103 single_machine.cc:361] Starting new session
I0000 00:00:1738548908.728638 18106103 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1738548908.728647 18106103 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
I0000 00:00:1738548912.245098 18106103 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1738548912.245120 18106103 pluggable_device_factory.cc:271] Created TensorFlow device (

In [7]:
outputNames = [output.name for output in onnxModel.graph.output]
outputNames

['dense']

## Inference

In [8]:
print(onnxModel.graph.input)
print('\n')
print(onnxModel.graph.output)

[name: "image"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__1508"
      }
      dim {
        dim_value: 224
      }
      dim {
        dim_value: 224
      }
      dim {
        dim_value: 3
      }
    }
  }
}
]


[name: "dense"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__1509"
      }
      dim {
        dim_value: 3
      }
    }
  }
}
]


In [9]:
testImage = cv2.imread("/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test/happy/2705.jpg_rotation_1.jpg")
testImage = cv2.resize(testImage, (CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]))
testImage = testImage.astype(np.float32)
testImage = np.expand_dims(testImage, axis=0)

In [11]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession("Models/EmotionDetectionViT.onnx", providers=providers)

onnxPred = m.run(outputNames, {onnxModel.graph.input[0].name: testImage})
print(onnxPred)
print(CONFIGURATION['CLASS_NAMES'][np.argmax(onnxPred[0])])

[array([[8.1243154e-05, 9.9953485e-01, 3.8389381e-04]], dtype=float32)]
happy


## Comparing Model's Latency

In [12]:
import time

In [23]:
startTime = time.time()
ViTModel(testImage)
endTime = time.time()
print("Keras Model Time on GPU: ", endTime - startTime)

print('\n')

startTime = time.time()
m.run(outputNames, {onnxModel.graph.input[0].name: testImage})
endTime = time.time()
print("ONNX Model Time on CPU: ", endTime - startTime)

Keras Model Time on GPU:  0.07996392250061035


ONNX Model Time on CPU:  0.0737910270690918
